Este modelo efetua o cálculo da discrepância entre os requisitos solicitados pela empresa e as qualificações apresentadas pelo candidato, gerando uma coluna denominada "Match" que varia de 0 a 1 para cada usuário e vaga disponível. Com base nesses valores de correspondência, o modelo identifica os melhores matches para um determinado usuário, recomendando-os. Adicionalmente, utiliza essas discrepâncias para criar gráficos que ilustram a proximidade das habilidades do usuário em relação às necessidades da empresa. Esses gráficos também destacam as lacunas entre o que a empresa exige e o que o candidato possui, fornecendo orientações sobre as áreas que o candidato pode aprimorar para aumentar suas chances de ingressar na empresa.

In [5]:
#Import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from math import pi
import plotly.express as px

In [ ]:
# Checar se o dataet de usuário está importando corretamente
datauser = pd.read_csv('NORM_JUNCAO_USER.csv')
datauser.head()


,Unnamed: 0,id,chave,lifestyle_classic,lifestyle_order,lifestyle_change,lifestyle_tireless,lifestyle_explorer,lifestyle_specialist,lifestyle_generalist,lifestyle_hybrid,key,superfit_dis,superfit_sin,superfit_cur,superfit_int,superfit_eng,superfit_res
0,0,1019062,14,0.294118,0.163265,0.222222,0.222222,0.234043,0.2,0.6,0.00,SF_DI,0.755556,0.711111,0.489362,0.760870,0.659091,0.571429
1,1,1019131,8,0.117647,0.102041,0.333333,0.200000,0.191489,0.0,0.6,0.25,SF_DR,0.688889,0.577778,0.446809,0.565217,0.522727,0.642857
2,2,1019138,14,0.176471,0.061224,0.222222,0.266667,0.361702,0.4,0.6,0.00,SF_IR,0.577778,0.533333,0.553191,0.717391,0.568182,0.595238
3,3,1017352,13,0.117647,0.102041,0.250000,0.155556,0.255319,0.8,0.0,0.00,SF_DS,0.711111,0.622222,0.489362,0.586957,0.386364,0.595238
4,4,1019110,14,0.176471,0.122449,0.194444,0.177778,0.255319,0.0,0.8,0.00,SF_DI,0.600000,0.533333,0.361702,0.630435,0.409091,0.357143


In [7]:
# Precisamos calcular o match entre um candidato e empresa, então o nosso modelo compara cada candidato com cada empresa e com base na diferença entre os valores de superfit e lifestyle ele calcula a compatiblidade entre os dois.
# Inserção dos Dados de Usuários/Empresas
datauser = pd.read_csv('NORM_JUNCAO_USER.csv')
datajob = pd.read_csv('NORM_JUNCAO_JOB.csv')
datauser.drop('Unnamed: 0', axis=1, inplace=True)
datajob.drop('Unnamed: 0', axis=1, inplace=True)
datauser.describe()
# Adição de uma coluna chamada "Match", que mostrará o quanto o candidato combina com a empresa
datauser['match'] = 1
datajob['match'] = 1
# Junção dos datasets
combinations = pd.merge(datauser, datajob, on='match')

# Features escolhidas: Todas as variáveis de superfit e lifestyle
# Justificativa: Como não foi fornecido um "peso" para cada variável (apenas o peso de superfit e lifestyle), foi decidido que todas as variáveis seriam utilizadas (inclusive acordado com o parceiro no dia 08/09)
df = combinations
mediasSuperfit = (abs(df['score_res'] - df['superfit_res']) + abs(df['score_eng'] - df['superfit_eng']) + abs(df['score_int'] - df['superfit_int']) + abs(df['score_cur'] - df['superfit_cur']) + abs(df['score_sin'] - df['superfit_sin']) + abs(df['score_dis'] - df['superfit_dis']))/6
mediasLifestyle = (abs(df['score_classic'] - df['lifestyle_classic']) + abs(df['score_order'] - df['lifestyle_order']) + abs(df['score_change'] - df['lifestyle_change']) + abs(df['score_tireless'] - df['lifestyle_tireless']) + abs(df['score_explorer'] - df['lifestyle_explorer']) + abs(df['score_specialist'] - df['lifestyle_specialist']) + abs(df['score_generalist'] - df['lifestyle_generalist']))/7
df['match'] = 1-(2* mediasSuperfit + mediasLifestyle)/3
# Na parte acima, é alavancada a coluna "match" com base na média das diferenças entre os scores do usuário e da empresa. Dessa forma será recomendado
def getmatches(id: int, number : int):
    # Filtrar o dataset para o id do usuário
    df_filtrado = df[df['id'] == id]
    # Ordenar o dataset pelo match
    topmatches = df_filtrado.nlargest(number, 'match')[['id', 'job_opportunity_id', 'match']]
    # Retornar o dataset com os melhores matches
    return topmatches

              id  job_opportunity_id     match
3877311  1055906                2909  0.950588
3877209  1055906                2744  0.888900
3877212  1055906                2888  0.886975
3877323  1055906                2894  0.875032
3877329  1055906                2918  0.872204
3877301  1055906                2890  0.869897
3877194  1055906                2781  0.868066
3877303  1055906                2884  0.864608
3877189  1055906                2666  0.864201
3877192  1055906                2692  0.864197


In [8]:
# Teste para checar se as vagas estão sendo corretamente recomendadas ao candidato
id = int(input("Digite o id do Candidato: "))
numero = int(input("Digite o número de vagas que deseja recomendar: "))
print(getmatches(id, numero))

Digite o id do Candidato: 1055906
Digite o número de vagas que deseja recomendar: 20
              id  job_opportunity_id     match
3877311  1055906                2909  0.950588
3877209  1055906                2744  0.888900
3877212  1055906                2888  0.886975
3877323  1055906                2894  0.875032
3877329  1055906                2918  0.872204
3877301  1055906                2890  0.869897
3877194  1055906                2781  0.868066
3877303  1055906                2884  0.864608
3877189  1055906                2666  0.864201
3877192  1055906                2692  0.864197
3877319  1055906                2957  0.862662
3877328  1055906                2962  0.861552
3877215  1055906                2679  0.860806
3877235  1055906                2922  0.852887
3877268  1055906                2801  0.851638
3877178  1055906                2761  0.849814
3877325  1055906                2960  0.848189
3877222  1055906                2785  0.846912
3877267  1055906      

In [9]:
# Pedir o id do usuário e do trabalho que o usuário tem o desejo de entrar e salvar em 2 variáveis, user_id e job_id
user_id = int(input('Digite o id do usuario: '))
job_id = int(input('Digite o id da empresa: '))

Digite o id do usuario: 1055906
Digite o id da empresa: 2909


In [10]:
# Calcular as distâncias entre usuário e empresa e salvar em novas variáveis, além de checar se o id de usuário e empresa existem.

# Ler os arquivos CSVs relacionados ao usuário e empresa
datauser = pd.read_csv('NORM_JUNCAO_USER.csv')
datajob = pd.read_csv('NORM_JUNCAO_JOB.csv')

# Remover coluna 'Unnamed: 0' de ambos DFs
datauser.drop('Unnamed: 0', axis=1, inplace=True)
datajob.drop('Unnamed: 0', axis=1, inplace=True)

# Reduzir ambos os DFs para apenas a linha do id do usuario e empresa escolhidos anteriormente
datajob = datajob[(datajob['job_opportunity_id'] == job_id)]
datauser = datauser[(datauser['id'] == user_id )]

# Se o id da empresa/usuário não forem encontrados retornar a mensagem abaixo, caso contrário retornar "Encontrado"
if datajob.empty or datauser.empty:
    print(f"Id de usuario {user_id} ou de empresa {job_id} não existem.")
else:
    print('Encontrado')

# Reiniciar o index do datajob e datauser afim dos indexes de ambos forem reduzidos a 0
datajob = datajob.reset_index(drop=True)
datauser = datauser.reset_index(drop=True)

# Calcular a distância entre o score do usuário e o score que a empresa pede
dRes = abs(datajob['score_res'] - datauser['superfit_res'])
dEng = abs(datajob['score_eng'] - datauser['superfit_eng'])
dInt = abs(datajob['score_int'] - datauser['superfit_int'])
dCur = abs(datajob['score_cur'] - datauser['superfit_cur'])
dSin = abs(datajob['score_sin'] - datauser['superfit_sin'])
dDis = abs(datajob['score_dis'] - datauser['superfit_dis'])


Encontrado


In [11]:
# Produzir o gráfico das distâncias entre o candidato e a vaga

# Selecionar as colunas do DF datajob a fim de serem usadas
dataj = ['score_res', 'score_eng', 'score_int', 'score_cur', 'score_sin', 'score_dis', 'score_classic', 'score_order', 'score_change',
         'score_tireless', 'score_explorer', 'score_specialist', 'score_generalist', 'job_opportunity_id']

# Selecionar as colunas do DF datauser a fim de sere usadas
datau = ['superfit_res', 'superfit_eng', 'superfit_int', 'superfit_cur', 'superfit_sin', 'superfit_dis', 'lifestyle_classic',
         'lifestyle_order', 'lifestyle_change', 'lifestyle_tireless', 'lifestyle_explorer', 'lifestyle_specialist', 'lifestyle_generalist', 'id']

# Transformar os DFs datajob e datauser para apenas as colunas especificadas
datajob = datajob[dataj]
datauser = datauser[datau]

# Transformar as distâncias de uma Series do pandas para float
dRes = dRes[0]
dEng = dEng[0]
dInt = dInt[0]
dCur = dCur[0]
dSin = dSin[0]
dDis = dDis[0]


# Criar um dicionário em forma de DF com o valor float das distâncias relacionado ao significado das mesmas
df = pd.DataFrame(dict(
    value = [dRes, dEng, dInt, dCur, dSin, dDis],
    variable = ['Resiliente', 'Engajado', 'Colaborativo', 'Original', 'Pragmático', 'Consistente']

))

# Plotar as linhas do gráfico
fig = px.line_polar(df, r = 'value', theta = 'variable', line_close = True)
fig.update_traces(fill = 'toself')

fig.update_layout(title=f"Distância do candidato aos requerimentos da empresa referentes ao usuário {user_id} e empresa {job_id}")

# Mostrar o gráfico
fig.show()

In [12]:
# Produzir o gráfico que mostra os valores que a empresa pede e os valores que o candidato possui.

# Ler os arquivos CSVs relacionados ao usuário e emprego
datauser = pd.read_csv('NORM_JUNCAO_USER.csv')
datajob = pd.read_csv('NORM_JUNCAO_JOB.csv')

# Remover coluna "Unnamed: 0" de ambos os DFs
datauser.drop('Unnamed: 0', axis=1, inplace=True)
datajob.drop('Unnamed: 0', axis=1, inplace=True)

# Transformar ambos os DFs em apenas uma linha relacionada ao job_id e user_id escolhidos
datajob = datajob[(datajob['job_opportunity_id'] == job_id)]
datauser = datauser[(datauser['id'] == user_id )]

# Resetar o index afim de ambos terem apenas o index = 0
datajob = datajob.reset_index(drop=True)
datauser = datauser.reset_index(drop=True)

# Filtrar o usuário pelo ID
datauser = datauser[datauser['id'] == user_id]

# Filtrar a empresa pelo ID da oportunidade de emprego
datajob = datajob[datajob['job_opportunity_id'] == job_id]


# Criar 2 listas para as colunas a fim de terem seus valores trocados de 0 para 0.01
columns_to_replace_user = ['superfit_res', 'superfit_eng', 'superfit_int', 'superfit_cur', 'superfit_sin', 'superfit_dis']
columns_to_replace_job = ['score_res', 'score_eng', 'score_int', 'score_cur', 'score_sin', 'score_dis']

# Trocar valores iguais a 0 por 0.01 afim de melhor observação no gráfico de radar
datauser[columns_to_replace_user] = np.where(datauser[columns_to_replace_user] == 0, 0.01, datauser[columns_to_replace_user])

datajob[columns_to_replace_job] = np.where(datajob[columns_to_replace_job] == 0, 0.01, datajob[columns_to_replace_job])



# Criar um dataframe com os respectivos dados relacionados ao usuario e empresa afim de plotar o grafico.
df_2 = pd.DataFrame({
    'value': [
        datauser.at[0, 'superfit_res'], datauser.at[0, 'superfit_eng'], datauser.at[0, 'superfit_int'],
        datauser.at[0, 'superfit_cur'], datauser.at[0, 'superfit_sin'], datauser.at[0, 'superfit_dis'],
        datajob.at[0, 'score_res'], datajob.at[0, 'score_eng'], datajob.at[0, 'score_int'],
        datajob.at[0, 'score_cur'], datajob.at[0, 'score_sin'], datajob.at[0, 'score_dis']
    ],
    'variable': [
        'Resiliência', 'Engajamento', 'Colaboração', 'Originalidade', 'Pragmatismo', 'Consistência',
        'Resiliência', 'Engajamento', 'Colaboração', 'Originalidade', 'Pragmatismo', 'Consistência'
    ],
    'Legenda': [
        'Usuário', 'Usuário', 'Usuário', 'Usuário', 'Usuário', 'Usuário',
        'Empresa', 'Empresa', 'Empresa', 'Empresa', 'Empresa', 'Empresa'
    ]
})

fig = px.line_polar(df_2, r = 'value', theta = 'variable', line_close = True,
                    color = 'Legenda')
fig.update_traces(fill = 'toself')

fig.show()

# dis - Consistente
# sin - Pragmático
# cur - Original
# int - Colaborativo
# eng - Engajado
# res - Resiliente

In [13]:
dResTs = datajob['score_res'] - datauser['superfit_res']
dEngTs = datajob['score_eng'] - datauser['superfit_eng']
dIntTs = datajob['score_int'] - datauser['superfit_int']
dCurTs = datajob['score_cur'] - datauser['superfit_cur']
dSinTs = datajob['score_sin'] - datauser['superfit_sin']
dDisTs = datajob['score_dis'] - datauser['superfit_dis']

dResTs = dResTs[0]
dEngTs = dEngTs[0]
dIntTs = dIntTs[0]
dCurTs = dCurTs[0]
dSinTs = dSinTs[0]
dDisTs = dDisTs[0]

# Criar um set para guardar os nomes das competencias a fim de remove-las
exclude_competencies = set()



# Checar se as distâncias são negativas e se forem, adicionar uma string ao set
if dResTs < 0:
    exclude_competencies.add('Resiliente')

if dEngTs < 0:
    exclude_competencies.add('Engajado')

if dIntTs < 0:
    exclude_competencies.add('Colaborativo')

if dCurTs < 0:
    exclude_competencies.add('Original')

if dSinTs < 0:
    exclude_competencies.add('Pragmático')

if dDisTs < 0:
    exclude_competencies.add('Consistente')

# Filtrar o DF por colunas com valor acima de 0.3
filtered_df = df[df['value'] >= 0.3]

# Pegar os nomes das variáveis que as linhas tem valor acima de 0.3
variable_names = filtered_df['variable'].tolist()

# Remover o nome das competencias se elas estao em exclude competencies
filtered_variable_names = [name for name in variable_names if name not in exclude_competencies]

# Checar se tem algo a recomendar
if filtered_variable_names:
    # Criar a mensagem de recomendação com as variaveis
    recommendation_message = f'É recomendado que você melhore nos seguintes quesitos: {", ".join(filtered_variable_names)}'

    # Printar a mensagem
    print(recommendation_message)
else:
    # Se não tiver variáveis, printar a mensagem abaixo
    print('Não há recomendações de melhoria.')


Não há recomendações de melhoria.
